## Import necesssary packages <a class="anchor" id="chap1"></a>

In [ ]:
import sys
import gc
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer
from skimage import util
from numba import cuda
import tensorflow as tf 
from caiman.motion_correction import MotionCorrect, tile_and_correct, motion_correction_piecewise
import caiman as cm

sys.path.insert(0, '/home/mestalbet/PythonScripts/Usiigaci/Mask R-CNN')
from mrcnn import model as modellib
from mrcnn import utils
from train import cellConfig

import operator
from operator import itemgetter
from itertools import groupby
from tqdm import tqdm
import random
import cv2
from PIL import Image
from tifffile import imsave,imread
from skimage import exposure
from skimage.color import rgb2gray
from skimage import io
import os
import numpy as np

import datetime
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import time
from glob import glob
import csv
import pandas as pd
import urllib.request

import matplotlib.pyplot as plt
import seaborn as sns
import colorsys
import plotly
import plotly.graph_objs as go
import plotly.plotly as py
import plotly.offline as offline
plotly.offline.init_notebook_mode(connected=True)

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
IOU_THRESHOLD = 0.6
OVERLAP_THRESHOLD = 0.8
MIN_DETECTIONS = 1

import imagej
ij = imagej.init('/home/mestalbet/Fiji.app')
import imglyb

## Read experiment directories <a class="anchor" id="chap2"></a>

In [ ]:
# files = []
# start_dir = "/home/mestalbet/bucket/BackupData/"
# pattern = "file.tif"

# for dir, _, _ in os.walk(start_dir):
#     files.extend(glob(os.path.join(dir, pattern)))

## Define functions for processing stacks <a class="anchor" id="chap3"></a>

In [ ]:
def alignstack(chosenpath):
    temppath = "/home/mestalbet/PythonScripts/deleteme/mc/"
    # maximum allowed rigid shift in pixels (view the movie to get a sense of motion)
    max_shifts = (24, 24)
    # create a new patch every x pixels for pw-rigid correction
    strides = (128, 128)
    # overlap between patches (size of patch strides+overlaps)
    overlaps = (48, 48)
    # length in frames of each chunk of the movie (to be processed in parallel)
    num_frames_split = 50
    # maximum deviation allowed for patch with respect to rigid shifts
    max_deviation_rigid = 3
    # flag for performing rigid or piecewise rigid motion correction
    pw_rigid = True  
    # flag for correcting motion using bicubic interpolation (otherwise FFT interpolation is used)
    shifts_opencv = True
    
    # replicate values along the boundary (if True, fill in with NaN)
    border_nan = 'copy'

    if 'dview' in locals():
        cm.stop_server(dview=dview)
    c, dview, n_processes = cm.cluster.setup_cluster(
        backend='local', n_processes=None, single_thread=False)

    mc = MotionCorrect(chosenpath, dview=dview, max_shifts=max_shifts,
                       strides=strides, overlaps=overlaps,
                       max_deviation_rigid=max_deviation_rigid,
                       shifts_opencv=shifts_opencv, nonneg_movie=True,
                       border_nan=border_nan, use_cuda=True)
    mc.motion_correct(save_movie=True)
    m_rig = cm.load(mc.mmap_file)
    img = np.asarray(m_rig)
    if 'dview' in locals():
        cm.stop_server(dview=dview)
    return(img)

#     macro = """ setBatchMode(true);
#                 run("Memory & Threads...", "maximum=10000 parallel=2");
#                 open("{}");
#                 run("8-bit");
#                 run("Z Project...", "projection=[Max Intensity]");
#                 run("moco ", "value=51 downsample_value=0 template=MAX_{} stack={} log=[Generate log file] plot=[No plot]");
#                 saveAs("Results", "{}");
#                 close();
#                 open("{}");
#                 run("Z Project...", "projection=[Max Intensity]");
#                 run("moco ", "value=51 downsample_value=0 template=MAX_{}-1.tif stack={}-1.tif log=[Choose log file] plot=[No plot] choose="{}");
#                 saveAs("Tiff", "{}");
#     """.format(chosenpath,
#                chosenpath.split('/')[-1], 
#                chosenpath.split('/')[-1],
#                '/'.join(chosenpath.split('/')[:-1]) + "/results.csv",
#                chosenpath,
#                chosenpath.split('/')[-1].split('.')[0], 
#                chosenpath.split('/')[-1].split('.')[0],
#               '/'.join(chosenpath.split('/')[:-1]) + "/results.csv",
#               '/'.join(chosenpath.split('/')[:-1]) + "/"+chosenpath.split('/')[-1].split('.')[0]+ "_aligned.tif")
#     print(macro)
#     ij.script().run('Macro.ijm', macro, True).get()
#     img = io.imread('/'.join(chosenpath.split('/')[:-1]) + "/"+chosenpath.split('/')[-1].split('.')[0]+ "_aligned.tif")
#     return(img)

def improve_raw(impath):
    macro = """open("{}");
               run("Normalize Local Contrast", "block_radius_x=40 block_radius_y=40 standard_deviations=3 center stretch");
               run("Enhance Local Contrast (CLAHE)", "blocksize=127 histogram=256 maximum=3 mask=*None*");
               run("Gaussian Blur 3D...", "x=2 y=2 z=2");
               saveAs("TIFF", "{}");
               run("Close All");""".format(impath.replace("/", "/"), impath.replace("/", "/").replace(".tif", "_clean.tif"))
    ij.script().run('Macro.ijm', macro, True).get()
    img = io.imread(impath.replace(".tif", "_clean.tif"))
    return(img)


def createmeanimage(alignedpath):
    rescale = MinMaxScaler(feature_range=(0, 1))
    pt = QuantileTransformer(output_distribution='normal')
    temppath = "D:\\PythonScripts\\deletme\\"
    outpath = os.path.join(temppath, "kf_temp.tif")
    if os.path.exists(outpath):
        os.remove(outpath)
    macro = """open("{}");
    run("Kalman Stack Filter", "acquisition_noise=0.05 bias=0.80");
    saveAs("Tiff", "{}");
    run("Close All");""".format(alignedpath.replace('\\', '/'), outpath.replace('\\', '/'))
    ij.script().run('Macro.ijm', macro, True).get()
    img = io.imread(outpath)
    img = img.mean(axis=0)
    gc.collect()
    img = pt.fit_transform(img)
    img = rescale.fit_transform(img)
    img = exposure.equalize_adapthist(img, clip_limit=0.05)
    imrescaled = cv2.resize(img, dsize=(1024, 1024),
                            interpolation=cv2.INTER_AREA)
    imrescaled = util.invert(imrescaled)
    imrescaled = imrescaled*(255**2)
    imrescaled = imrescaled.astype('u2')
    finalpath = os.path.join(temppath, "meanimage.tif")
    if os.path.exists(finalpath):
        os.remove(finalpath)
    io.imsave(finalpath, imrescaled)
    imclean = improve_raw(finalpath)
    return(imclean)

def cleanliorimage(alignedpath):
    rescale = MinMaxScaler(feature_range=(0, 1))
    pt = QuantileTransformer(output_distribution='normal')
    img = io.imread(alignedpath)
    img = pt.fit_transform(img)
    img = rescale.fit_transform(img)
    img = exposure.equalize_adapthist(img, clip_limit=0.05)
    imrescaled = cv2.resize(img, dsize=(1024, 1024),
                            interpolation=cv2.INTER_AREA)
    imrescaled = util.invert(imrescaled)
    imrescaled = imrescaled*(255**2)
    imrescaled = imrescaled.astype('u2')
    io.imsave(os.path.join(alignedpath,"inference/"), imrescaled)
    return(imrescaled)

# get a cell
def getcell(mask, cellnum):
    cell = mask == cellnum
    return(cell.astype("float32"))


def random_colors(N):
    """
    Generate brewer colors.
    To get visually distinct colors
    """
    colors = sns.color_palette("deep", N)
    return(np.asarray(colors))


def apply_mask(image, mask, color, alpha=0.3):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c],
                                  image[:, :, c])
    return(image)


def createoverlay(meanimg, mask):
    font = cv2.FONT_HERSHEY_SIMPLEX
    img = cv2.bitwise_not(meanimg)
    img = np.dstack([img/(255**2), img/(255**2), img/(255**2)])
    numcells = len(np.unique(mask))
    colors = random_colors(numcells)
    for i in range(1, numcells-1):
        cellmask = getcell(mask, i)
        x = int(np.round(np.mean(np.where(cellmask > 0)[1])))
        y = int(np.round(np.mean(np.where(cellmask > 0)[0])))
        img = apply_mask(img, cellmask, colors[i])
        img = cv2.putText(img, str(
            i-1), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255**2, 0), 2, cv2.LINE_AA)
    return(img)


def gettraces(frames, mask):
    numcells = len(np.unique(mask))
    traces = []
    for cellid in range(1, numcells-1):
        cellmask = getcell(mask, cellid)
        cellmask[cellmask == 0 ] = np.nan
        traces.append([np.nanmean(np.multiply(frame.astype("float32"), cv2.resize(
            cellmask, (256, 256)))) for frame in frames])
    sig = pd.DataFrame({"cells": range(1, numcells-1), "traces": traces})
    return(sig) #has a bug - it's averaging with many zeros


def plotTrace(signals, cellnum):
    offline.iplot([go.Scatter(x=np.arange(
        len(signals['traces'].iloc[cellnum])), y=signals['traces'].iloc[cellnum])])

## Function for calling the trained neural network <a class="anchor" id="chap4"></a>

In [ ]:
def segmentimage(data_dir, model_list):
    class Logger(object):
        def __init__(self):
            self.terminal = sys.stdout
#                 self.log = open(data_dir+"log.log", "a")

        def write(self, message):
            self.log = open(data_dir+"log.log", "a")
            self.terminal.write(message)
            self.log.write(message)
            self.log.close()

        def flush(self):
            pass

    class ImageDataset(utils.Dataset):
        def load_images(self, dataset_dir):
            """
            Loads dataset images.
            :param dataset_dir: string, path to dataset directory.
            :return: None
            """
            self.add_class("cell", 1, "cell")

            image_ids = [fn for fn in os.listdir(dataset_dir)
                         if any(fn.endswith(ext) for ext in ['tif', "png"])]

            for image_id in image_ids:
                self.add_image(
                    'cell',
                    image_id=os.path.splitext(image_id)[0],
                    path=os.path.join(dataset_dir, image_id)
                )

    class CellInferenceConfig(cellConfig):
        # Set batch size to 1 to run one image at a time
        GPU_COUNT = 1
        IMAGES_PER_GPU = 1
        # Don't resize imager for inferencing
        IMAGE_RESIZE_MODE = "pad64"
        # Non-max suppression threshold to filter RPN proposals.
        # You can increase this during training to generate more propsals.
        RPN_NMS_THRESHOLD = 0.7
        # define the folder path to data for prediction
        global data_dir
        all_files = []
        sub_directory = []
        for root, dirs, files in os.walk(data_dir):
            for file in files:
                relativePath = os.path.relpath(root, data_dir)
                if relativePath == ".":
                    relativePath = ""
                all_files.append(
                    (relativePath.count(os.path.sep), relativePath, file))
        all_files.sort(reverse=True)
        for (count, folder), files in groupby(all_files, itemgetter(0, 1)):
            sub_directory.append(folder)

    def detect(model, data_dir, out_dir):
        '''
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
        detection_dir = "detections_{:%Y%m%dT%H%M%S}".format(datetime.datetime.now())
        detection_dir = os.path.join(out_dir, detection_dir)
        os.makedirs(detection_dir)
        '''
        # Read dataset
        dataset = ImageDataset()
        dataset.load_images(data_dir)
        dataset.prepare()
        # Load over images
        for image_id in tqdm(dataset.image_ids):
            # Load image and run detection
            image = dataset.load_image(image_id)
            # Detect objects
            r = model.detect([image], verbose=0)[0]
            # Encode image to RLE. Returns a string of multiple lines
            source_id = dataset.image_info[image_id]["id"]

            #out_path = os.path.join(detection_dir, '%s.png' % str(source_id))
            out_path = os.path.join(out_dir, '%s.png' % str(source_id))

            mask = np.argmax(r['masks'], 2)
            cv2.imwrite(os.path.normpath(out_path), mask)

    def compute_iou(mask1, mask2):
        """
        Computes Intersection over Union score for two binary masks.
        :param mask1: numpy array
        :param mask2: numpy array
        :return:
        """
        intersection = np.sum((mask1 + mask2) > 1)
        union = np.sum((mask1 + mask2) > 0)

        return intersection / float(union)

    def compute_overlap(mask1, mask2):
        intersection = np.sum((mask1 + mask2) > 1)

        overlap1 = intersection / float(np.sum(mask1))
        overlap2 = intersection / float(np.sum(mask2))
        return overlap1, overlap2

    def sort_mask_by_cells(mask, min_size=50):
        """
        Returns size of each cell.
        :param mask:
        :return:
        """
        cell_num = np.unique(mask)
        cell_sizes = [(cell_id, len(np.where(mask == cell_id)[0]))
                      for cell_id in cell_num if cell_id != 0]

        cell_sizes = [x for x in sorted(
            cell_sizes, key=lambda x: x[1], reverse=True) if x[1 > min_size]]

        return cell_sizes

    def merge_multiple_detections(masks):
        """

        :param masks:
        :return:
        """
        cell_counter = 0
        final_mask = np.zeros(masks[0].shape)

        masks_stats = [sort_mask_by_cells(mask) for mask in masks]
        cells_left = sum([len(stats) for stats in masks_stats])

        while cells_left > 0:
            # Choose the biggest cell from available
            cells = [stats[0][1] if len(
                stats) > 0 else 0 for stats in masks_stats]
            reference_mask = cells.index(max(cells))

            reference_cell = masks_stats[reference_mask].pop(0)[0]

            # Prepare binary mask for cell chosen for comparison
            cell_location = np.where(masks[reference_mask] == reference_cell)

            cell_mask = np.zeros(final_mask.shape)
            cell_mask[cell_location] = 1

            masks[reference_mask][cell_location] = 0

            # Mask for storing temporary results
            tmp_mask = np.zeros(final_mask.shape)
            tmp_mask += cell_mask

            for mask_id, mask in enumerate(masks):
                # For each mask left
                if mask_id != reference_mask:
                    # # Find overlapping cells on other masks
                    overlapping_cells = list(np.unique(mask[cell_location]))

                    try:
                        overlapping_cells.remove(0)
                    except ValueError:
                        pass

                    # # If only one overlapping, check IoU and update tmp mask if high
                    if len(overlapping_cells) == 1:
                        overlapping_cell_mask = np.zeros(final_mask.shape)
                        overlapping_cell_mask[np.where(
                            mask == overlapping_cells[0])] = 1

                        iou = compute_iou(cell_mask, overlapping_cell_mask)
                        if iou >= IOU_THRESHOLD:
                            # Add cell to temporary results and remove from stats and mask
                            tmp_mask += overlapping_cell_mask
                            idx = [i for i, cell in enumerate(
                                masks_stats[mask_id]) if cell[0] == overlapping_cells[0]][0]
                            masks_stats[mask_id].pop(idx)
                            mask[np.where(mask == overlapping_cells[0])] = 0

                    # # If more than one overlapping check area overlapping
                    elif len(overlapping_cells) > 1:
                        overlapping_cell_masks = [
                            np.zeros(final_mask.shape) for _ in overlapping_cells]

                        for i, cell_id in enumerate(overlapping_cells):
                            overlapping_cell_masks[i][np.where(
                                mask == cell_id)] = 1

                        for cell_id, overlap_mask in zip(overlapping_cells, overlapping_cell_masks):
                            overlap_score, _ = compute_overlap(
                                overlap_mask, cell_mask)

                            if overlap_score >= OVERLAP_THRESHOLD:
                                tmp_mask += overlap_mask

                                mask[np.where(mask == cell_id)] = 0
                                idx = [i for i, cell in enumerate(masks_stats[mask_id])
                                       if cell[0] == cell_id][0]
                                masks_stats[mask_id].pop(idx)

                    # # If none overlapping do nothing

            if len(np.unique(tmp_mask)) > 1:
                cell_counter += 1
                final_mask[np.where(tmp_mask >= MIN_DETECTIONS)] = cell_counter

            cells_left = sum([len(stats) for stats in masks_stats])

        bin_mask = np.zeros(final_mask.shape)
        bin_mask[np.where(final_mask > 0)] = 255

        cv2.imwrite(os.path.join(data_dir, 'results/final_bin.png'), bin_mask)
        cv2.imwrite(os.path.join(data_dir, 'results/final.png'), final_mask)
        return(final_mask)

    def process_sequence(masks):
        """

        :param masks:
        :return:
        """

    def postprocess(data_dir, out_dir):
        """

        :param data_dir:
        :param out_dir:
        :return:
        """
        #os.makedirs(out_dir, exist_ok=True)
        models_dir = [os.path.join(data_dir, filename)
                      for filename in os.listdir(data_dir)]
        print('Merging multiple models predictions.')
        filenames = os.listdir(models_dir[0])

        for filename in tqdm(filenames):
            masks = [cv2.imread(os.path.join(model_dir, filename), 0)
                     for model_dir in models_dir]

            result = merge_multiple_detections(masks)
            bin_result = np.zeros(result.shape)
            bin_result[np.where(result > 0)] = 255

            cv2.imwrite(os.path.join(out_dir, filename), result)
    all_files = []
    sub_directory = []
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            relativePath = os.path.relpath(root, data_dir)
            if relativePath == ".":
                relativePath = ""
            all_files.append(
                (relativePath.count(os.path.sep), relativePath, file))
    all_files.sort(reverse=True)
    for (count, folder), files in groupby(all_files, itemgetter(0, 1)):
        sub_directory.append(folder)

    config = CellInferenceConfig()
    model = modellib.MaskRCNN(
        mode="inference", config=config, model_dir=data_dir)
    sys.stdout = Logger()

    total_start = time.time()
    run_time_log = []
    for i in sub_directory:
        counter = 1
        start = time.time()
        mask_duplicate_dir = os.path.join(data_dir, i+'_mask')
        for m in model_list:
            # print(m)
            print('>Loading model from: ', m)
            model.load_weights(m, by_name=True)
            try:
                predict_location = os.path.join(data_dir, i)
                print('prediction for: ', predict_location)
                print('model run '+str(counter)+' of '+str(len(model_list)))
                try:
                    out_dir = os.path.join(
                        mask_duplicate_dir, "_"+str(counter))
                    os.makedirs(out_dir)
                except:
                    print('failed to create mask folder')
                try:
                    detect(model, predict_location, out_dir)
                except:
                    print('failed to deploy the inference, skipping...')
            except:
                print('error, skipping...')
            counter += 1
        try:
            avg_prediction_dir = os.path.join(data_dir, i+"_mask_avg")
            os.makedirs(avg_prediction_dir)
        except:
            print('failed to create avg mask folder')
        postprocess(mask_duplicate_dir, avg_prediction_dir)
        end = time.time()
        time_diff = end-start
        run_time_log.append(time_diff)
        hour = time_diff // 3600
        time_diff %= 3600
        minutes = time_diff // 60
        time_diff %= 60
        seconds = time_diff
        print('prediction run time = %d hr: %d min: %d s' %
              (hour, minutes, seconds))
    print(run_time_log)
    for item in run_time_log:
        runtimelogfile = open('exptime.txt', 'w')
        runtimelogfile.write("%s\n" % item)
        runtimelogfile.close()
    total_end = time.time()
    total_time = total_end - total_start
    total_day = total_time // (3600*24)
    total_time %= (3600*24)
    total_hour = total_time // 3600
    total_time %= 3600
    total_minutes = total_time // 60
    total_time %= 60
    total_seconds = total_time
    print('Total prediction run time = %d day: %d hr: %d min: %d s' %
          (total_day, total_hour, total_minutes, total_seconds))

## Align stack and create denoised mean image <a class="anchor" id="chap6"></a>

In [ ]:
# test = "/home/mestalbet/PythonScripts/Tiff_stacks/file1.tif"
# test ="/home/mestalbet/bucket/BackupData/180326/3/file.tif"
# alignedstack = alignstack(test)
from tensorflow.python.lib.io import file_io
with file_io.FileIO('gs://segproj/BackupData/180326/3/file.tif', mode='r') as input_f:
    alignedstack = alignstack(input_f)

In [ ]:
plt.imshow(np.mean(alignedstack,axis=0))

In [ ]:
%%capture
model_path_1 = "/home/mestalbet/PythonScripts/out/latest/mask_rcnn_cell_0100.h5"
model_path_2 = "/home/mestalbet/PythonScripts/out/latest/mask_rcnn_cell_0198.h5"
model_path_3 = "/home/mestalbet/PythonScripts/out/latest/mask_rcnn_cell_0199.h5"
model_path_4 = "/home/mestalbet/PythonScripts/out/latest/mask_rcnn_cell_0200.h5"
model_list = [model_path_1, model_path_2, model_path_3, model_path_4]

experimentids = list(set([file.split('/')[5] for file in files]))
experimentids = sorted(experimentids)

for file in files:
    gc.collect()
    outpath = "/home/mestalbet/PythonScripts/Results_LiorImages/"
    tempdir = file.split("/")
    chosenpath = "_".join([tempdir[-3],tempdir[-2],tempdir[-1]])
    alignpath = os.path.join(outpath, chosenpath.replace(".tif", "_aligned.tif"))
    meanpath = os.path.join('/'.join(alignpath.split("/")[:-1]), "inference/")
    if not os.path.exists(meanpath):
        os.makedirs(meanpath)
    meanpath = os.path.join(meanpath, alignpath.replace(
        "_aligned.tif", "_meanimage.tif").split("/")[-1])
    # Process Chosen Data File
    alignedstack = alignstack(file)
    imsave(alignpath, alignedstack.astype('int16'), imagej=True)
    meanimage = createmeanimage(alignpath)
    io.imsave(meanpath, meanimage)
# Run Mask RCNN Inference
data_dir = "/".join(meanpath.split("/")[:-1])+"/"
gc.collect()
segmentimage(data_dir, model_list)

In [ ]:
%%capture
outpath = "/home/mestalbet/PythonScripts/Results_LiorImages/"
alignedpaths = [os.path.join(outpath,f) for f in os.listdir(outpath) if f.endswith("tif")]
for alignedpath in alignedpaths:
    meanimage = createmeanimage(alignedpath)
    gc.collect()
    meanpath = os.path.join('/'.join(alignedpath.split("/")[:-1]), "inference/")
    meanpath = os.path.join(meanpath, alignedpath.replace("_aligned.tif", "_meanimage.tif").split("\\")[-1])
    io.imsave(meanpath, meanimage)

In [ ]:
# from joblib import Parallel, delayed
# outpath = "/home/mestalbet/PythonScripts/Results_LiorImages/"
# alignedpaths = [os.path.join(outpath,f) for f in os.listdir(outpath) if f.endswith("tif")]
# Parallel(n_jobs=2)(delayed(cleanliorimage)(alignedpath) for alignedpath in alignedpaths)

In [ ]:
%%capture
outpath = "/home/mestalbet/PythonScripts/Results_LiorImages/"
data_dir = os.path.join(outpath,"inference/")
model_path_1 = "/home/mestalbet/PythonScripts/out/latest/mask_rcnn_cell_0100.h5"
model_path_2 = "/home/mestalbet/PythonScripts/out/latest/mask_rcnn_cell_0198.h5"
model_path_3 = "/home/mestalbet/PythonScripts/out/latest/mask_rcnn_cell_0199.h5"
model_path_4 = "/home/mestalbet/PythonScripts/out/latest/mask_rcnn_cell_0200.h5"
model_list = [model_path_1, model_path_2, model_path_3, model_path_4]
gc.collect()
segmentimage(data_dir, model_list)

In [ ]:
# outpath = "/home/mestalbet/PythonScripts/Results_LiorImages/"
# data_dir = os.path.join(outpath,"inference/")
# meanfiles = [os.path.join(data_dir,f) for f in os.listdir(data_dir) if f.endswith("tif")]
# model_path_1 = "/home/mestalbet/PythonScripts/out/latest/mask_rcnn_cell_0100.h5"
# model_path_2 = "/home/mestalbet/PythonScripts/out/latest/mask_rcnn_cell_0198.h5"
# model_path_3 = "/home/mestalbet/PythonScripts/out/latest/mask_rcnn_cell_0199.h5"
# model_path_4 = "/home/mestalbet/PythonScripts/out/latest/mask_rcnn_cell_0200.h5"
# model_list = [model_path_1, model_path_2, model_path_3, model_path_4]
# gc.collect()

# class CellInferenceConfig(cellConfig):
#         # Set batch size to 1 to run one image at a time
#         GPU_COUNT = 1
#         IMAGES_PER_GPU = 1
#         # Don't resize imager for inferencing
#         IMAGE_RESIZE_MODE = "pad64"
#         # Non-max suppression threshold to filter RPN proposals.
#         # You can increase this during training to generate more propsals.
#         RPN_NMS_THRESHOLD = 0.7
#         # define the folder path to data for prediction
#         global data_dir
#         all_files = []
#         sub_directory = []
#         for root, dirs, files in os.walk(data_dir):
#             for file in files:
#                 relativePath = os.path.relpath(root, data_dir)
#                 if relativePath == ".":
#                     relativePath = ""
#                 all_files.append(
#                     (relativePath.count(os.path.sep), relativePath, file))
#         all_files.sort(reverse=True)
#         for (count, folder), files in groupby(all_files, itemgetter(0, 1)):
#             sub_directory.append(folder)

# class ImageDataset(utils.Dataset):
#     def load_images(self, dataset_dir):
#         """
#         Loads dataset images.
#         :param dataset_dir: string, path to dataset directory.
#         :return: None
#         """
#         self.add_class("cell", 1, "cell")

#         image_ids = [fn for fn in os.listdir(dataset_dir)
#                      if any(fn.endswith(ext) for ext in ['tif', "png"])]

#         for image_id in image_ids:
#             self.add_image(
#                 'cell',
#                 image_id=os.path.splitext(image_id)[0],
#                 path=os.path.join(dataset_dir, image_id)
#             )

# def merge_multiple_detections(masks):
#     """

#     :param masks:
#     :return:
#     """
#     cell_counter = 0
#     final_mask = np.zeros(masks[0].shape)

#     masks_stats = [sort_mask_by_cells(mask) for mask in masks]
#     cells_left = sum([len(stats) for stats in masks_stats])

#     while cells_left > 0:
#         # Choose the biggest cell from available
#         cells = [stats[0][1] if len(
#             stats) > 0 else 0 for stats in masks_stats]
#         reference_mask = cells.index(max(cells))

#         reference_cell = masks_stats[reference_mask].pop(0)[0]

#         # Prepare binary mask for cell chosen for comparison
#         cell_location = np.where(masks[reference_mask] == reference_cell)

#         cell_mask = np.zeros(final_mask.shape)
#         cell_mask[cell_location] = 1

#         masks[reference_mask][cell_location] = 0

#         # Mask for storing temporary results
#         tmp_mask = np.zeros(final_mask.shape)
#         tmp_mask += cell_mask

#         for mask_id, mask in enumerate(masks):
#             # For each mask left
#             if mask_id != reference_mask:
#                 # # Find overlapping cells on other masks
#                 overlapping_cells = list(np.unique(mask[cell_location]))

#                 try:
#                     overlapping_cells.remove(0)
#                 except ValueError:
#                     pass

#                 # # If only one overlapping, check IoU and update tmp mask if high
#                 if len(overlapping_cells) == 1:
#                     overlapping_cell_mask = np.zeros(final_mask.shape)
#                     overlapping_cell_mask[np.where(
#                         mask == overlapping_cells[0])] = 1

#                     iou = compute_iou(cell_mask, overlapping_cell_mask)
#                     if iou >= IOU_THRESHOLD:
#                         # Add cell to temporary results and remove from stats and mask
#                         tmp_mask += overlapping_cell_mask
#                         idx = [i for i, cell in enumerate(
#                             masks_stats[mask_id]) if cell[0] == overlapping_cells[0]][0]
#                         masks_stats[mask_id].pop(idx)
#                         mask[np.where(mask == overlapping_cells[0])] = 0

#                 # # If more than one overlapping check area overlapping
#                 elif len(overlapping_cells) > 1:
#                     overlapping_cell_masks = [
#                         np.zeros(final_mask.shape) for _ in overlapping_cells]

#                     for i, cell_id in enumerate(overlapping_cells):
#                         overlapping_cell_masks[i][np.where(
#                             mask == cell_id)] = 1

#                     for cell_id, overlap_mask in zip(overlapping_cells, overlapping_cell_masks):
#                         overlap_score, _ = compute_overlap(
#                             overlap_mask, cell_mask)

#                         if overlap_score >= OVERLAP_THRESHOLD:
#                             tmp_mask += overlap_mask

#                             mask[np.where(mask == cell_id)] = 0
#                             idx = [i for i, cell in enumerate(masks_stats[mask_id])
#                                    if cell[0] == cell_id][0]
#                             masks_stats[mask_id].pop(idx)

#                 # # If none overlapping do nothing

#         if len(np.unique(tmp_mask)) > 1:
#             cell_counter += 1
#             final_mask[np.where(tmp_mask >= MIN_DETECTIONS)] = cell_counter

#         cells_left = sum([len(stats) for stats in masks_stats])
#     return(final_mask)

# model = modellib.MaskRCNN(mode="inference", config=CellInferenceConfig(), model_dir=data_dir)
# model.load_weights(model_path_1, by_name=True)            
# dataset = ImageDataset()
# dataset.load_images(data_dir)
# dataset.prepare()
# from joblib import Parallel, delayed
# test = []
# def detect(i,dataset = dataset, model = model):
#     image = dataset.load_image(dataset.image_ids[i])
#     r = model.detect([image], verbose=0)[0]
#     return(np.argmax(r['masks'], 2))
# test = Parallel(n_jobs=8)(delayed(detect)(j) for j in range(0,len(dataset.image_ids)))

In [ ]:
plt.imshow(mask)

In [ ]:
def extractsignals(m):
    meanpath = os.path.join('/'.join(m.split("/")[:-2]), m.split("/")[-1].replace(".png",".tif"))
    stackpath = os.path.join('/'.join(meanpath.split("/")[:3]),meanpath.split("/")[-1].replace("_meanimage.tif","_aligned.tif"))
    csvpath = stackpath.replace("_aligned.tif", "_extracted_traces.csv")
    mask = io.imread(m)
    meanimage = io.imread(meanpath)
    img = createoverlay(meanimage, mask.astype('uint16'))
    img = img*(255*255)
    img = img.astype("uint16")
    io.imsave(meanpath.replace(".tif", "_overlay.png"),img)
    alignedstack = imread(stackpath)                        
    signals = gettraces(alignedstack, mask)
    signals.to_csv(csvpath)

In [ ]:
outpath = "/home/mestalbet/PythonScripts/Results_LiorImages/"
mask_dir = os.path.join(data_dir,"_mask_avg")
mean_dir = os.path.join(data_dir,"inference/")
# meanpaths = [os.path.join(outpath,f) for f in os.listdir(outpath) if f.endswith("tif")]
# alignedpaths = [os.path.join(outpath,f) for f in os.listdir(outpath) if f.endswith("tif")]
maskfiles = [os.path.join(mask_dir,f) for f in os.listdir(mask_dir) if f.endswith("png")]

for m in maskfiles:
    meanpath = os.path.join('/'.join(m.split("/")[:-2]), m.split("/")[-1].replace(".png",".tif"))
    stackpath = os.path.join('/'.join(meanpath.split("/")[:3]),meanpath.split("/")[-1].replace("_meanimage.tif","_aligned.tif"))
    csvpath = stackpath.replace("_aligned.tif", "_extracted_traces.csv")
    mask = io.imread(m)
    meanimage = io.imread(meanpath)
    img = createoverlay(meanimage, mask.astype('uint16'))
    img = img*(255*255)
    img = img.astype("uint16")
    io.imsave(meanpath.replace(".tif", "_overlay.png"),img)
    alignedstack = imread(stackpath)                        
    signals = gettraces(alignedstack, mask)
    signals.to_csv(csvpath)